In [ ]:
api_key ="sk-None-" 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

file_path = "D:\\muthukumar\\ocr\\extracted_text.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_content = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_text(text_content)
documents = [Document(page_content=chunk, metadata={"source": file_path}) for chunk in text_chunks]

def prompt_function(documents, api_key):
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    vectorstore = FAISS.from_documents(documents, embedding=embeddings)
    retriever = vectorstore.as_retriever()

    prompt = PromptTemplate.from_template(
        """
        <s> [INST] You are an assistant for question-answering tasks. Use the following pieces of retrieved context 
        to answer the question. If you don't know the answer, just say that you don't know. Use three sentences
        maximum and keep the answer concise. If there is previous context provided, incorporate it into your response. [/INST] </s> 
        [INST] Previous context: {previous_context}
        Question: {input}
        Context: {context}
        Answer: [/INST]
        """
    )

    llm = ChatOpenAI(
        model_name="gpt-4o-mini",
        openai_api_key=api_key,
        max_retries=10
    )

    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)
    return retrieval_chain

api_key = api_key  # Replace with  API key
retrieval_chain = prompt_function(documents, api_key)

previous_context = ""

while True:
    question = input("Enter your question (or type 'exit' to quit): ")
    if question.lower() == "exit":
        break
    
    response = retrieval_chain.invoke({"input": question, "previous_context": previous_context})
    
    print(response['answer'])
    previous_context += f"\n{response}"  


C:\Users\murugan\AppData\Local\Temp\ipykernel_35448\3607154471.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)
C:\Users\murugan\AppData\Local\Temp\ipykernel_35448\3607154471.py:38: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


The first client name mentioned is Teleah Grand. She is associated with the patient Buckshot, a French Bulldog. This information was retrieved from the Animal Care Center at Stonebridge Ranch records.
The first client phone number mentioned is 214-563-1688. This number is listed as the emergency contact for Teleah Grand. She is associated with the patient Buckshot, a French Bulldog.
The first patient name mentioned is Buckshot. Buckshot is a French Bulldog owned by Teleah Grand. This information is recorded in the Animal Care Center at Stonebridge Ranch medical chart.
The only patient name mentioned in the provided context is Buckshot. Buckshot is a French Bulldog owned by Teleah Grand. There are no additional patient names listed in the retrieved context.
I don't know.
